# Build Dataset - Kanji by stroke order 0x01

Actually the current implementation won't sort the kanji. But i like them to be sorted, when they are presented to the user.

In [ ]:
import json
import os

from jamdict import Jamdict

In [ ]:
jamdict = Jamdict()

In [ ]:
radicals = list(jamdict.radk.keys())


Since we now now the raicals, we will now go through a loop, fetch every kanji for it, check, whether we already know it, if not, then we will lookup it and then get the stroke order. from the chars array.

In [ ]:
processed_kanji = set()
for radical in radicals:
    print("Processing {}".format(radical))
    kanjifor_radical = set(jamdict.radk[radical])
    # print(kanjifor_radical)
    processed_kanji = processed_kanji | kanjifor_radical
    print(len(processed_kanji))

In [ ]:
kanji_strokes = {}
kanji_freq = {}

processed_kanji = set()
for radical in radicals:
    print("Processing {}".format(radical))
    kanjifor_radical = jamdict.radk[radical]
    print("found: "+str(len(kanjifor_radical)) +" kanji.")

    for kanji in kanjifor_radical:
        if kanji in processed_kanji:
            continue
        
        # print("Kanji {}".format(kanji))
        result=jamdict.lookup(kanji,strict_lookup=True)
        
        for char in result.chars:
            char_dict = char.to_json()
            kanji_strokes[char_dict['literal']]=char_dict['stroke_count']
        
        frequency = int(result.chars[0].to_json()['freq'])
        if(frequency==0):
            frequency=10000
        kanji_freq[char_dict['literal']]=frequency
            
        processed_kanji.add(kanji)
    
    
    #processed_kanji = processed_kanji | kanjifor_radical


In [ ]:
print(len(kanji_strokes))
print(len(kanji_freq))


In [ ]:
tree = {}

max_strokes = max(kanji_strokes.values())
# init the stroke_tree
for i in range(1+max_strokes):
    tree[i]=list()

for kanji, strokes in kanji_strokes.items():
    tree[strokes].append(kanji)
    # but what is the right criteria in the kanji stroke domain? (frequency?), i would opt for frequency.
    # tree[strokes].sort()

In [ ]:
tree

In [ ]:
max(kanji_strokes.values())

In [ ]:
def kf(k):
    if not k in kanji_freq:
        return 10000
    return int(kanji_freq[k])

for strokes in tree.keys():
    kanjilist = tree[strokes]
    sorted_list = sorted(kanjilist, key=lambda kanji: kf(kanji), reverse=False)
    tree[strokes] = sorted_list

In [ ]:
tree

In [ ]:
kanji_freq

In [ ]:
index = list(tree.keys())


flattened = []
for i in index:
    flattened.extend(tree[i])

flattened

In [ ]:
DATA_DIR = '../data'
RADICAL_STROKE_DATA = 'kanjiStrokeData.json'

with open(os.path.join(DATA_DIR,RADICAL_STROKE_DATA),'w') as outputFile:
    json.dump( {'tree': tree, 'list':flattened }, outputFile);